In [8]:
import numpy as np

from brodatz.data_utils import process_file, process_file_some_classes, read_some_classes, save_model, read_train_test_sets, save_history
# from brodatz.utils import probas_to_classes, accuracy, curr_date, try_args_generator, try_args, get_bottlebeck_features
from brodatz.utils import *
from brodatz.visualization_utils import plot_image, plot, plot_history, plot_confusion_matrix

from brodatz.models.keras_neural_net import KerasNeuralNetwork

from brodatz.brodatz_generator import *

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

from keras import regularizers, optimizers, utils
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
# remove prefixes
import os
path = '/mnt/82db778e-0496-450c-9b25-d1e50a90e476/data/data4stas/01_data_cls'
os.chdir(path)

In [4]:
train_filename = 'Alexander_cls_train.csv'
test_filenames = ['Alexander_cls_test.csv',
                  'Alexander_cls_test1.csv',
                  'Alexander_cls_test2.csv',
                  'Alexander_cls_test3.csv',
                  'Alexander_cls_test4.csv']
classes_range = np.arange(0, 111)
X_train, y_train, num_classes, X_val, y_val, test_list = read_train_test_sets(train_filename, test_filenames, classes_range)

In [5]:
X_val, y_val = test_list[0][:2]

In [15]:
batch_size = 64
nb_train_samples = 2000
nb_validation_samples = 1000

train_generator = brodatz_generator_3_channels(X_train, y_train, batch_size=batch_size)
val_generator = brodatz_generator_3_channels(X_val, y_val, batch_size=batch_size)

bottleneck_features_train, bottleneck_features_validation = get_bottlebeck_features(train_generator, val_generator, batch_size, nb_train_samples, nb_validation_samples)

In [10]:
for i in range(len(test_list)):
    test_list[i][0] = np.vstack((test_list[i][0].T,)*3).T

In [6]:
X_train = np.vstack((X_train.T,)*3).T
print(X_train.shape)

X_val = np.vstack((X_val.T,)*3).T
print(X_val.shape)

(555, 200, 200, 3)
(2220, 200, 200, 3)


In [11]:
rotation_ranges = [0, 45, 90, 135, 180]
shear_ranges = [0, 0.2, 0.25, 1/3, 0.5, 0.75, 1]
shift_ranges = [0, 0.15, 0.3, 0.4, 0.45]
horizontal_flips = [True, False]
vertical_flips = [True, False]
zoom_ranges = [0, 0.2, 0.35, 0.5, 0.7, [0.6, 0.7], [1.4, 1.5]]
fill_modes = ['reflect', 'nearest', 'wrap']
learning_rates = [0.001, 0.002, 0.005, 0.01]
regularization_strengths = [0.001, 0.01, 0.05, 0.1]

rotation_range = [0, 0, 0, 180]
shear_range = 0
shift_range = [0.5, 0.4, 0.3, 0.4]
horizontal_flip = True
vertical_flip = True
zoom_range = [0, [0.55, 0.65], [1.45, 1.55], 0]
fill_mode = 'reflect'
batch_size = 256
epochs = 500
learning_rate = 0.002
regularization_strength = 0.01

input_shape = X_train[0].shape
regularizer = regularizers.l2(regularization_strength)
optimizer = optimizers.SGD(lr=learning_rate, decay=0, momentum=0.9, nesterov=True)

net = KerasNeuralNetwork(input_shape, num_classes, regularizer=regularizer, optimizer=optimizer)
net.compile_model_finetuning_2()

datagen = brodatz_generator(X_train, y_train, batch_size)

steps_per_epoch = len(X_train) / batch_size * 1

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)
callbacks = [reduce_lr]
callbacks = None

# try_args_fit(bottleneck_features_train, y_train, num_classes, X_val, y_val, test_list, net, optimizer, callbacks, rotation_range, shear_range, shift_range, horizontal_flip, vertical_flip, zoom_range, fill_mode, batch_size, epochs, learning_rate, regularization_strength, save_model=False, save_results=False)

try_args_generator(X_train, y_train, num_classes, X_val, y_val, test_list, net, optimizer, datagen, steps_per_epoch, callbacks, rotation_range, shear_range, shift_range, horizontal_flip, vertical_flip, zoom_range, fill_mode, batch_size, epochs, learning_rate, regularization_strength)

# for rotation_range in rotation_ranges:
#     try_args(rotation_range, shear_range, shift_range, horizontal_flip, vertical_flip, zoom_range, fill_mode, batch_size, epochs)

# for shear_range in shear_ranges:
#     try_args(rotation_range, shear_range, shift_range, horizontal_flip, vertical_flip, zoom_range, fill_mode, batch_size, epochs)

Epoch 1/500
3/2 [=========================================] - 14s 5s/step - loss: 1.1525 - acc: 0.9440 - val_loss: 1.2082 - val_acc: 0.9194
Epoch 2/500
3/2 [=========================================] - 9s 3s/step - loss: 1.1345 - acc: 0.9583 - val_loss: 1.2029 - val_acc: 0.9225
Epoch 3/500
3/2 [=========================================] - 9s 3s/step - loss: 1.1250 - acc: 0.9506 - val_loss: 1.1977 - val_acc: 0.9248
Epoch 4/500
3/2 [=========================================] - 9s 3s/step - loss: 1.1479 - acc: 0.9479 - val_loss: 1.2018 - val_acc: 0.9243
Epoch 5/500
3/2 [=========================================] - 9s 3s/step - loss: 1.1081 - acc: 0.9557 - val_loss: 1.1971 - val_acc: 0.9257
Epoch 6/500
3/2 [=========================================] - 9s 3s/step - loss: 1.1471 - acc: 0.9506 - val_loss: 1.1920 - val_acc: 0.9261
Epoch 7/500
3/2 [=========================================] - 9s 3s/step - loss: 1.1257 - acc: 0.9557 - val_loss: 1.1989 - val_acc: 0.9207
Epoch 8/500
3/2 [=========

3/2 [=========================================] - 10s 3s/step - loss: 1.0412 - acc: 0.9635 - val_loss: 1.1196 - val_acc: 0.9279
Epoch 60/500
3/2 [=========================================] - 10s 3s/step - loss: 1.0798 - acc: 0.9370 - val_loss: 1.1208 - val_acc: 0.9297
Epoch 61/500
3/2 [=========================================] - 10s 3s/step - loss: 1.0386 - acc: 0.9661 - val_loss: 1.1255 - val_acc: 0.9230
Epoch 62/500
3/2 [=========================================] - 10s 3s/step - loss: 1.0610 - acc: 0.9596 - val_loss: 1.1242 - val_acc: 0.9234
Epoch 63/500
3/2 [=========================================] - 10s 3s/step - loss: 1.0391 - acc: 0.9573 - val_loss: 1.1167 - val_acc: 0.9239
Epoch 64/500
3/2 [=========================================] - 10s 3s/step - loss: 1.0440 - acc: 0.9531 - val_loss: 1.1144 - val_acc: 0.9261
Epoch 65/500
3/2 [=========================================] - 10s 3s/step - loss: 1.0442 - acc: 0.9609 - val_loss: 1.1166 - val_acc: 0.9270
Epoch 66/500
3/2 [========

3/2 [=========================================] - 10s 3s/step - loss: 1.0035 - acc: 0.9520 - val_loss: 1.0529 - val_acc: 0.9293
Epoch 118/500
3/2 [=========================================] - 10s 3s/step - loss: 0.9641 - acc: 0.9596 - val_loss: 1.0584 - val_acc: 0.9284
Epoch 119/500
3/2 [=========================================] - 10s 3s/step - loss: 0.9861 - acc: 0.9544 - val_loss: 1.0575 - val_acc: 0.9261
Epoch 120/500
3/2 [=========================================] - 10s 3s/step - loss: 0.9930 - acc: 0.9635 - val_loss: 1.0481 - val_acc: 0.9297
Epoch 121/500
3/2 [=========================================] - 10s 3s/step - loss: 0.9748 - acc: 0.9648 - val_loss: 1.0463 - val_acc: 0.9315
Epoch 122/500
3/2 [=========================================] - 10s 3s/step - loss: 0.9879 - acc: 0.9609 - val_loss: 1.0499 - val_acc: 0.9257
Epoch 123/500
3/2 [=========================================] - 10s 3s/step - loss: 0.9848 - acc: 0.9581 - val_loss: 1.0468 - val_acc: 0.9297
Epoch 124/500
3/2 [=

Epoch 175/500
3/2 [=========================================] - 10s 3s/step - loss: 0.9215 - acc: 0.9635 - val_loss: 0.9957 - val_acc: 0.9293
Epoch 176/500
3/2 [=========================================] - 10s 3s/step - loss: 0.9328 - acc: 0.9544 - val_loss: 0.9949 - val_acc: 0.9266
Epoch 177/500
3/2 [=========================================] - 10s 3s/step - loss: 0.9254 - acc: 0.9631 - val_loss: 0.9894 - val_acc: 0.9311
Epoch 178/500
3/2 [=========================================] - 10s 3s/step - loss: 0.9056 - acc: 0.9674 - val_loss: 0.9898 - val_acc: 0.9311
Epoch 179/500
3/2 [=========================================] - 10s 3s/step - loss: 0.9237 - acc: 0.9544 - val_loss: 0.9934 - val_acc: 0.9252
Epoch 180/500
3/2 [=========================================] - 10s 3s/step - loss: 0.9184 - acc: 0.9610 - val_loss: 0.9907 - val_acc: 0.9275
Epoch 181/500
3/2 [=========================================] - 10s 3s/step - loss: 0.9235 - acc: 0.9544 - val_loss: 0.9881 - val_acc: 0.9279
Epoch 

Epoch 233/500
3/2 [=========================================] - 10s 3s/step - loss: 0.8653 - acc: 0.9570 - val_loss: 0.9368 - val_acc: 0.9333
Epoch 234/500
3/2 [=========================================] - 10s 3s/step - loss: 0.8563 - acc: 0.9703 - val_loss: 0.9385 - val_acc: 0.9302
Epoch 235/500
3/2 [=========================================] - 10s 3s/step - loss: 0.8433 - acc: 0.9609 - val_loss: 0.9400 - val_acc: 0.9284
Epoch 236/500
3/2 [=========================================] - 10s 3s/step - loss: 0.8663 - acc: 0.9661 - val_loss: 0.9367 - val_acc: 0.9315
Epoch 237/500
3/2 [=========================================] - 10s 3s/step - loss: 0.8737 - acc: 0.9680 - val_loss: 0.9348 - val_acc: 0.9315
Epoch 238/500
3/2 [=========================================] - 10s 3s/step - loss: 0.8585 - acc: 0.9596 - val_loss: 0.9376 - val_acc: 0.9288
Epoch 239/500
3/2 [=========================================] - 10s 3s/step - loss: 0.8725 - acc: 0.9583 - val_loss: 0.9391 - val_acc: 0.9293
Epoch 

Epoch 291/500
3/2 [=========================================] - 10s 3s/step - loss: 0.8002 - acc: 0.9641 - val_loss: 0.8887 - val_acc: 0.9365
Epoch 292/500
3/2 [=========================================] - 10s 3s/step - loss: 0.8313 - acc: 0.9648 - val_loss: 0.8885 - val_acc: 0.9351
Epoch 293/500
3/2 [=========================================] - 10s 3s/step - loss: 0.8414 - acc: 0.9479 - val_loss: 0.8881 - val_acc: 0.9338
Epoch 294/500
3/2 [=========================================] - 10s 3s/step - loss: 0.8116 - acc: 0.9645 - val_loss: 0.8897 - val_acc: 0.9311
Epoch 295/500
3/2 [=========================================] - 10s 3s/step - loss: 0.7975 - acc: 0.9661 - val_loss: 0.8960 - val_acc: 0.9261
Epoch 296/500
3/2 [=========================================] - 10s 3s/step - loss: 0.8254 - acc: 0.9583 - val_loss: 0.8919 - val_acc: 0.9302
Epoch 297/500
3/2 [=========================================] - 10s 3s/step - loss: 0.8273 - acc: 0.9520 - val_loss: 0.8880 - val_acc: 0.9333
Epoch 

Epoch 349/500
3/2 [=========================================] - 10s 3s/step - loss: 0.7568 - acc: 0.9648 - val_loss: 0.8543 - val_acc: 0.9266
Epoch 350/500
3/2 [=========================================] - 10s 3s/step - loss: 0.7906 - acc: 0.9570 - val_loss: 0.8494 - val_acc: 0.9329
Epoch 351/500
3/2 [=========================================] - 10s 3s/step - loss: 0.7447 - acc: 0.9699 - val_loss: 0.8474 - val_acc: 0.9333
Epoch 352/500
3/2 [=========================================] - 10s 3s/step - loss: 0.7794 - acc: 0.9622 - val_loss: 0.8477 - val_acc: 0.9374
Epoch 353/500
3/2 [=========================================] - 10s 3s/step - loss: 0.7573 - acc: 0.9714 - val_loss: 0.8443 - val_acc: 0.9383
Epoch 354/500
3/2 [=========================================] - 10s 3s/step - loss: 0.7669 - acc: 0.9695 - val_loss: 0.8481 - val_acc: 0.9360
Epoch 355/500
3/2 [=========================================] - 10s 3s/step - loss: 0.7733 - acc: 0.9714 - val_loss: 0.8548 - val_acc: 0.9288
Epoch 

Epoch 407/500
3/2 [=========================================] - 10s 3s/step - loss: 0.7514 - acc: 0.9518 - val_loss: 0.8147 - val_acc: 0.9324
Epoch 408/500
3/2 [=========================================] - 10s 3s/step - loss: 0.7434 - acc: 0.9635 - val_loss: 0.8144 - val_acc: 0.9342
Epoch 409/500
3/2 [=========================================] - 10s 3s/step - loss: 0.7408 - acc: 0.9596 - val_loss: 0.8113 - val_acc: 0.9329
Epoch 410/500
3/2 [=========================================] - 10s 3s/step - loss: 0.7367 - acc: 0.9661 - val_loss: 0.8103 - val_acc: 0.9338
Epoch 411/500
3/2 [=========================================] - 10s 3s/step - loss: 0.7168 - acc: 0.9709 - val_loss: 0.8109 - val_acc: 0.9342
Epoch 412/500
3/2 [=========================================] - 10s 3s/step - loss: 0.7332 - acc: 0.9648 - val_loss: 0.8117 - val_acc: 0.9351
Epoch 413/500
3/2 [=========================================] - 10s 3s/step - loss: 0.7255 - acc: 0.9701 - val_loss: 0.8114 - val_acc: 0.9369
Epoch 

Epoch 465/500
3/2 [=========================================] - 10s 3s/step - loss: 0.7185 - acc: 0.9528 - val_loss: 0.7773 - val_acc: 0.9378
Epoch 466/500
3/2 [=========================================] - 10s 3s/step - loss: 0.6904 - acc: 0.9766 - val_loss: 0.7801 - val_acc: 0.9369
Epoch 467/500
3/2 [=========================================] - 10s 3s/step - loss: 0.7034 - acc: 0.9661 - val_loss: 0.7831 - val_acc: 0.9342
Epoch 468/500
3/2 [=========================================] - 10s 3s/step - loss: 0.7012 - acc: 0.9686 - val_loss: 0.7827 - val_acc: 0.9311
Epoch 469/500
3/2 [=========================================] - 10s 3s/step - loss: 0.7113 - acc: 0.9596 - val_loss: 0.7848 - val_acc: 0.9270
Epoch 470/500
3/2 [=========================================] - 10s 3s/step - loss: 0.7112 - acc: 0.9609 - val_loss: 0.7816 - val_acc: 0.9311
Epoch 471/500
3/2 [=========================================] - 10s 3s/step - loss: 0.6999 - acc: 0.9618 - val_loss: 0.7768 - val_acc: 0.9342
Epoch 

In [40]:
net = KerasNeuralNetwork(input_shape, num_classes, regularizer=regularizer, optimizer=optimizer)
net.compile_model_finetuning()

In [41]:
net.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 200, 200, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 200, 200, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 200, 200, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 100, 100, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 100, 100, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 100, 100, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 50, 50, 128)       0         
__________

In [ ]:
rotation_ranges = [0, 45, 90, 135, 180]
shear_ranges = [0, 0.2, 0.25, 1/3, 0.5, 0.75, 1]
shift_ranges = [0, 0.15, 0.3, 0.4, 0.45]
horizontal_flips = [True, False]
vertical_flips = [True, False]
zoom_ranges = [0, 0.2, 0.35, 0.5, 0.7, [0.6, 0.7], [1.4, 1.5]]
fill_modes = ['reflect', 'nearest', 'wrap']
learning_rates = [0.001, 0.002, 0.005, 0.01]
regularization_strengths = [0.001, 0.01, 0.05, 0.1]

rotation_range = 180
shear_range = 0
shift_range = 0.4
horizontal_flip = True
vertical_flip = True
zoom_range = [0.5, 1.6]
fill_mode = 'reflect'

batch_size = 64
epochs = 1000
learning_rate = 0.002
regularization_strength = 0.01

input_shape = X_train[0].shape
regularizer = regularizers.l2(regularization_strength)
optimizer = optimizers.SGD(lr=learning_rate, decay=1e-6, momentum=0.9, nesterov=True)

net = KerasNeuralNetwork(input_shape, num_classes, regularizer=regularizer, optimizer=optimizer)
net.compile_model_3()

datagen = ImageDataGenerator(
    rotation_range=rotation_range,
    width_shift_range=shift_range,
    height_shift_range=shift_range,
    horizontal_flip=horizontal_flip,
    vertical_flip=vertical_flip,
    zoom_range=zoom_range,
    fill_mode=fill_mode)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

try_args(X_train, y_train, num_classes, X_val, y_val, test_list, net, optimizer, datagen, rotation_range, shear_range, shift_range, horizontal_flip, vertical_flip, zoom_range, fill_mode, batch_size, epochs, learning_rate, regularization_strength)

# try_args_generator(X_train, y_train, num_classes, X_val, y_val, test_list, net, optimizer, datagen, steps_per_epoch, [reduce_lr], rotation_range, shear_range, shift_range, horizontal_flip, vertical_flip, zoom_range, fill_mode, batch_size, epochs, learning_rate, regularization_strength)

# for rotation_range in rotation_ranges:
#     try_args(rotation_range, shear_range, shift_range, horizontal_flip, vertical_flip, zoom_range, fill_mode, batch_size, epochs)

# for shear_range in shear_ranges:
#     try_args(rotation_range, shear_range, shift_range, horizontal_flip, vertical_flip, zoom_range, fill_mode, batch_size, epochs)

In [32]:
net.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_40 (Conv2D)           (None, 196, 196, 32)      832       
_________________________________________________________________
activation_28 (Activation)   (None, 196, 196, 32)      0         
_________________________________________________________________
max_pooling2d_40 (MaxPooling (None, 98, 98, 32)        0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 96, 96, 64)        18496     
_________________________________________________________________
activation_29 (Activation)   (None, 96, 96, 64)        0         
_________________________________________________________________
max_pooling2d_41 (MaxPooling (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 46, 46, 128)       73856     
__________